<a href="https://www.kaggle.com/kamaljp/analysing-indiancompany-data?scriptVersionId=86801103" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install dataset

## What to Expect in this Notebook ?

Have you wondered how Reliance Industries is [performing](#Q7) based on its market value and earning per share? Or, have thought about the [powerful people](#Q2) behind multi-million INR companies in India who decide where share holder's profits are invested? Where are the companies in the universe of Finance?

How about the link between [Earnings and Market value](#Q6) or [Earnings to Book Value](#Q8). Not just these questions get answered by the data. The note book [shows how the data can be retrived](#start) from the websites that we visit regularly. 

All this is made possible with BeautifulSoup and the html links. All you need is the workhorse [functions](#fun). The majority of the challenge occurs in getting the data, and [upserting](#ups) into the database. [dataset](#fun) library does heavy lifting of locating the correct cell in the database to insert. No SQL Query!!! 

# [And much more.. hop on](#questions)

most of the cells are hidden, so the visual information can be directly presented. The entire notebook can be copied, and the code can be understood. That is how, I have learnt to code or just to cut, copy and Paaaasssttteee...

## <a id="content"> Contents </a>

### Purpose :

Datascience begins with Data. Data in this world starts with websites, and APIs. This notebook shows the complete cycle of data gathering to analysis conclusion. Emphasis is on Data gathering using the BeautifulSoup and Requests libraries. As I went through the extraction process, an idea struck. What will happen once the web 3.0 kicks in??? 

Explore the connection of the directors, board members in the Indian companies. This will give the information on the power centers and decision makers in the Indian organisations. To begin with, we will concentrate on the index stocks and then the move to more companies and their directors.

### Major Libraries Involved:

BeautifulSoup, Requests, Plotly Express and Dataset libraries

<a id="questions"></a>

With the data of the directors, board members in hand here are some questions that can be answered.

[What is the maximum number of Board members a company can have?](#Q1)

[Which board members are there in board of multiple companies?](#Q2)

[Which companies these board members are part of?](#Q3)

[How many type of Board members are there?](#Q4) 

[Are the number of board members and the market capitalisation linked?](#Q5)

[Is the Earnings Per Share linked to market capitalisation linked?](#Q6)

[Is the Book Value linked to market capitalisation linked?](#Q7)

[Is the Book Value linked to Earnings per Share? ](#Q8)


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import requests
import dataset
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse, urldefrag
from joblib import Parallel, delayed
import os
import plotly.express as px
import plotly.graph_objects as go

In [3]:
print(os.popen("curl https://www.moneycontrol.com/robots.txt").read())

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

User-Agent: *
Disallow: /intuit/promo/
Disallow: /network18/index.html
Disallow: /budget2013/live-union-budget.html
Disallow: /1039154/
Disallow: /tech_charts/
Disallow: /livefeed_pdf/
Disallow: /profile/
Disallow: /pdffiles/
Disallow: /news_html_files/
Disallow: /markets/market-stats/the-great-sensex-challenge/
Disallow: /markets/market-stats/the-great-sensex-challenge/winners/
Disallow: /printpage/
Disallow: /news/printpage/
Disallow: /news/business/economy/exclusive-india-exported-nearly-9300-metric-tonnes-of-oxygen-in-fy21-as-pandemic-rolled-on-6792711.html
Disallow: /open_in_app.php

Sitemap: https://mmb.moneycontrol.com/sitemap/stock_topic_sitemap.xml
Sitemap: https://mmb.moneycontrol.com/sitemap/market_view_topic_sitemap.xml
Sitemap: https://mmb.moneycontrol.com/sitemap/personal_finance_topic_sitemap.xml
Sitemap: https://mmb.moneycontrol.com/sitemap/latest_thread_sitemap.xml
Sitemap: https://www.moneycontrol.com/news/videos-sitemap-2020-index.xml
Sitemap: https://www.moneycontro

100  1277  100  1277    0     0   1444      0 --:--:-- --:--:-- --:--:--  1444


<a id="start"></a>
There are locations in the site which is disallowed for the crawlers. Let us be cautious, and ensure not to enter the restricted area.

In [4]:
#Instantiate the database for company owners

db = dataset.connect('sqlite:///owners.db')

source_url = 'https://www.moneycontrol.com/india/stockpricequote/'

In [5]:
session = requests.Session()
r = session.get(source_url)
#Read the complete webpage and parse it with html.parser
html_soup = BeautifulSoup(r.text, 'html.parser')

In [6]:
comp_link = html_soup.find_all(class_='pcq_tbl MT10')
comp_link = comp_link[0].find_all('td')
#The above class was located by checking the Elements tab in Dev tools in the Browser 
links = []
#Get the links of the 50 companies first
for company in comp_link:
    links.append(company.find('a').get('href'))

In [7]:
#Sanity checking the links that are scraped before using it for next step
base_url = 'https://www.moneycontrol.com/company-facts/'
def full_link(base_url,url):
    if url is None:
        return None
    link_split = url.split('/')
    full_url = base_url+link_split[-2]+'/management/'+link_split[-1]
    #print('Inserting {} company and its link {} into owners database'.format(link_split[-2],full_url))
    #creating companies table and inserting data inside it
    org_id = db['companies'].insert({'url': full_url,'entry_url': url,'company': link_split[-2]})
    
#Populating the Owner's companies table
for link in links:
    full_link(base_url,link)

In [8]:
company_link = pd.read_sql_table('companies','sqlite:///owners.db')
print('There are total {} companies in the database'.format(company_link.shape[0]))

There are total 471 companies in the database


[Back to contents](#content)

### <a id="fun"> Work horse functions </a>

In [9]:
def get_data(company,link):
    #Initiate new session for the companies
    comp = requests.Session()
    comp.headers.update({'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Mobile Safari/537.36',
                           'Referer': 'https://www.moneycontrol.com/promo/mc_wap_interstitial_dfp.php?classic=true'})
    r = comp.get(link)
    comp_soup = BeautifulSoup(r.text, 'html.parser')
    tab = comp_soup.find_all('table',class_='commomtable')
    for data in tab: #There are 2 elements in the tab tag
        name = data.select('tr > td')
        data_len = len(name)
        index = 0

        for i in range(int(data_len/2)): #There are minimum 8 elements in each of the tab's element
            db['mgmt'].insert({'name': data.select('tr > td')[index].get_text().strip(),
                               'designation': data.select('tr > td')[index+1].get_text().strip(),
                               'company': company})
            index += 2

In [10]:
def fund_data(company,entry_link):
    #Initiate new session for the companies
    comp = requests.Session()
    comp.headers.update({'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Mobile Safari/537.36'})
    r = comp.get(entry_link)
    fund_soup = BeautifulSoup(r.text, 'html.parser')
    o_table = fund_soup.find_all(class_='overvw_mob')
    if o_table is None:
        print('No Data Available from the webpage, so filling 0')
        db['companies'].upsert({'company':company,
                                'Market_Cap': float(0),
                                'EPS': float(0),
                                'PtoE': float(0),
                                'Book_value': float(0),
                                'PtoB': float(0)},['id'])

    else:
        data = o_table[0].select('li > div')
        #print('inserting data from overvw_mob class')
        db['companies'].upsert({'company':company,'Market_Cap': data[6].get_text(),
                               'EPS': data[11].get_text(),
                               'PtoE': data[12].get_text(),
                               'Book_value': data[14].get_text(),
                               'PtoB': data[15].get_text()},['company'])

### <a id="col"> Data Collector routines, that are marked down for notebook execution</a>

Lets cut to the Chase and work on the Data directly </a>

We won't need to scrap the data so often, so we will create choice during the execution to get the user input.

In [11]:
print('Do you want to scrape for new data, say Y/N?: Choosing N(change it to Y for re-scraping)')

scrape = 'N'

Do you want to scrape for new data, say Y/N?: Choosing N(change it to Y for re-scraping)


In [12]:
if scrape == 'Y':
    print('Starting to Scrape new data')
    for id in range(company_link.shape[0]):

        #print('inserting data of {} company into the owners database'.format(company_link.company[id]))

        get_data(company_link.company[id],company_link.url[id])

        #sleep(randint(30, 45))#adding a gap half a minute to 45 seconds to avoid continued hammering  

    print('Collecting Management data completed')

    for id in range(company_link.shape[0]):

        #print('inserting data of {} company into the owners database'.format(company_link.company[id]))

        fund_data(company_link.company[id],company_link.entry_url[id])

        #print(company_link.company[id])

        #sleep(randint(30, 45))#adding a gap half a minute to 45 seconds to avoid continued hammering  

    print('Collecting Fundamental data completed')
else:
    os.chdir('../input/indian-companies-fundamentaldata')
    print('reading companies table to company_link dataset')
    company_link = pd.read_sql_table('companies','sqlite:///owners.db')
    print('reading mgmt table to management dataset')
    management = pd.read_sql_table('mgmt','sqlite:///owners.db')

reading companies table to company_link dataset
reading mgmt table to management dataset


[Back to contents](#content)

[Back to questions](#questions)

### <a id="Q1"> 1) What is the maximum number of Board members a company can have?</a>

In [13]:
#Loading the management table as dataset
q1 = management.groupby('company')['name'].count().reset_index()
q1.sort_values(by='name',ascending=False,inplace=True)
print('{} has maximum number of Board member and it is {}'.format(q1.company.values[0],q1.name.values[0]))

vis1 = px.bar(data_frame=q1[:10],y='company',x='name',color='company')
vis1.update_layout(title='Company with the highest number of board members')
vis1.show()

larsentoubro has maximum number of Board member and it is 18


[Back to contents](#content)

[Back to questions](#questions)

### <a id="Q2"> Which Board members are there in more than one company can have?</a> 

In [14]:
q2 = management.groupby('name')['id'].count().reset_index()
q2.sort_values(by='id',ascending=False,inplace=True)
q2 = q2[q2.name != '']
print('There are {} members who are part of multiple companies'.format(q2[q2.id > 1].shape[0]))
vis2 = px.bar(data_frame=q2[:10],y='name',x='id',color='name')
vis2.update_layout(title='Directors who are part of more than one company')
vis2.show()

There are 485 members who are part of multiple companies


[Back to contents](#content)

[Back to questions](#questions)

### <a id="Q3"> Which companies these board members are part of?</a> 

In [15]:
dir_list = q2[q2.id > 1].name.values
#create dictionary to hold the director names, and their companies
multiple_comp = {}
for director in dir_list:
    #create the list of companies and attach the list to the director name  
    multiple_comp[director] = management.loc[management.name == director,'company'].values

#Write the dictionary to dataframe and display. Since the dictionary is having different length values, 
director_comp = pd.DataFrame(list(multiple_comp.values()),index=multiple_comp.keys())
director_comp.head()

,0,1,2,3,4,5,6
N Chandrasekaran,indianhotelscompany,tatachemicals,tataconsumerproducts,tatamotors,tatasteel,tataconsultancyservices,tatamotorsdvr
Haigreve Khaitan,ceat,inoxleisure,jswsteel,mahindramahindra,techmahindra,torrentpharmaceuticals,None
Kumar Mangalam Birla,adityabirlacapital,adityabirlafashionretail,centurytextilesindustries,grasimindustries,hindalcoindustries,ultratechcement,None
Anil Mukim,gujaratnarmadavalleyfertilizerschemicals,gujaratstatefertilizerschemicals,gujaratalkalieschemicals,gujaratstatepetronet,gujaratgas,None,None
Shikha Sharma,ambujacements,drreddyslaboratories,mahindramahindra,tataconsumerproducts,techmahindra,None,None


[Back to contents](#content)

[Back to questions](#questions)

### <a id="Q4"> How many types of board members are there, what is their distribution?</a> 

In [16]:
q4 = management.groupby('designation')['company'].count().reset_index()
q4.sort_values(by='company',inplace=True,ascending=False)

vis4 = px.bar(data_frame=q4[:10],y='designation',x='company',color='designation')
vis4.update_layout(title='Types of Board members and their distribution')
vis4.show()

[Back to contents](#content)

[Back to questions](#questions)

### <a id="Q5"> Are the number of board members and the market capitalisation linked? </a> 

In [17]:
def rem_comma(x):
    if x != 0.0:
        return x.replace(',','')
    else:
        pass

In [18]:
#Need to change the datatypes of company_link dataframe, and the data needs to be cleaned
company_link.loc[company_link.PtoE == '--','PtoE'] = 0.0
company_link.loc[company_link.PtoB == '--','PtoB'] = 0.0
company_link.loc[company_link.EPS == '--','EPS'] = 0.0
company_link.loc[company_link.Market_Cap == '--','Market_Cap'] = 0.0

company_link.loc[:,'PtoE'] = company_link.PtoE.apply(lambda x : rem_comma(x))
company_link.loc[:,'PtoB'] = company_link.PtoB.apply(lambda x : rem_comma(x))
company_link.loc[:,'EPS'] = company_link.EPS.apply(lambda x : rem_comma(x))
company_link.loc[:,'Market_Cap'] = company_link.Market_Cap.apply(lambda x : rem_comma(x))

company_link['PtoB'] = company_link['PtoB'].astype('float')
company_link['EPS'] = company_link['EPS'].astype('float')
company_link['PtoE'] = company_link['PtoE'].astype('float')
company_link['Market_Cap'] = company_link['Market_Cap'].astype('float')

In [19]:
company_link = pd.merge(left=company_link,right=q1,left_on='company',right_on='company',how='left')
company_link.columns = ['id', 'url', 'entry_url', 'company', 'Market_Cap', 'EPS', 'PtoE',
                        'Book_value', 'PtoB','Board_count']


In [20]:
vis5 = px.scatter(data_frame=company_link,x='Market_Cap',y='Board_count',
                  color='Market_Cap',color_continuous_scale=px.colors.cyclical.Phase_r,
                 size_max=25,size = 'Board_count',text='company',title="layout.hovermode='x'")
vis5.update_layout(title='Board Count and Market Capitalisation',height=1000)
vis5.update_traces(mode="markers", hovertemplate=None)
vis5.update_xaxes(title="Market Capitalisation Cr INR",type='log')
vis5.show()

[Back to contents](#content)

[Back to questions](#questions)

### <a id="Q6"> Is the Earnings Per Share linked to market capitalisation linked? </a> 

In [21]:
vis6 = px.scatter(data_frame=company_link,x='Market_Cap',y='EPS',
                  color='Market_Cap',color_continuous_scale=px.colors.cyclical.Phase_r,
                 size_max=25,size = 'Board_count',text='company',title="layout.hovermode='x'")
vis6.update_layout(title='EPS vs Market Capitalisation',height=1000)
vis6.update_traces(mode="markers", hovertemplate=None)
vis6.update_xaxes(title="Market Capitalisation Cr INR",type='log')
vis6.update_yaxes(title="EPS ratio",type='log')
vis6.show()

[Back to contents](#content)

[Back to questions](#questions)

### <a id="Q7"> Is the Book Value linked to market capitalisation linked? </a> 

In [22]:
vis7 = px.scatter(data_frame=company_link,x='Market_Cap',y='Book_value',
                  color='Market_Cap',color_continuous_scale=px.colors.cyclical.Phase_r,
                 size_max=25,size = 'Board_count',text='company',title="layout.hovermode='x'")
vis7.update_layout(title='Book Value per share vs Market Capitalisation',height=1000)
vis7.update_traces(mode="markers", hovertemplate=None)
vis7.update_xaxes(title="Market Capitalisation Cr INR",type='log')
vis7.update_yaxes(title="Book value per Share")
vis7.show()

[Back to contents](#content)

[Back to questions](#questions)

### <a id="Q8"> Is the Book Value linked to Earnings per Share? </a> 

In [23]:
vis8 = px.scatter(data_frame=company_link,x='EPS',y='Book_value',
                  color='EPS',color_continuous_scale=px.colors.cyclical.Phase_r,
                 size_max=25,size = 'Board_count',text='company',title="layout.hovermode='x'")
vis8.update_layout(title='Book Value per share vs Earning Per Share',height=1000)
vis8.update_traces(mode="markers", hovertemplate=None)
vis8.update_xaxes(title="Earning Per Share",type='log')
vis8.update_yaxes(title="Book value per Share")
vis8.show()

[Back to contents](#content)

[Back to questions](#questions)

Data and the visuals raises more question than they answer. The data has much more hidden inside. Appropriate feature engineering might bring more information. The questions which are creative and provocative make it more interesting. Thanks for joining this journey.